In [16]:
%load_ext autoreload
%autoreload 2
from causalboundingengine.algorithms.tianpearl import TianPearl
from causalboundingengine.algorithms.manski import Manski
from causalboundingengine.scenarios import BinaryConf
from causalboundingengine.scenarios import ContIV
from causalboundingengine.scenarios import BinaryIV
import numpy as np
import logging
logging.basicConfig(level=logging.WARNING)

In [16]:
import numpy as np
import pandas as pd
from causalboundingengine.scenarios import BinaryConf
import time

def generate_dataset(n, confounding_strength=0.5, seed=None):
    if seed is not None:
        np.random.seed(seed)

    U = np.random.binomial(1, 0.5, n)
    X = np.random.binomial(1, 0.3 + confounding_strength * U, n)
    Y = np.random.binomial(1, 0.2 + 0.3 * X + confounding_strength * U, n)

    return X, Y

def analyze_dataset(X, Y, dataset_id):
    scenario = BinaryConf(X, Y)
    algorithms = ['manski', 'tianpearl', 'autobound']

    results = []
    for alg_name in algorithms:
        start_time = time.time()
        try:
            alg_func = getattr(scenario.ATE, alg_name)
            bounds = alg_func()
            end_time = time.time()

            results.append({
                'dataset_id': dataset_id,
                'algorithm': alg_name,
                'lower_bound': bounds[0],
                'upper_bound': bounds[1],
                'width': bounds[1] - bounds[0],
                'computation_time': end_time - start_time,
                'success': True
            })
        except Exception as e:
            results.append({
                'dataset_id': dataset_id,
                'algorithm': alg_name,
                'lower_bound': None,
                'upper_bound': None,
                'width': None,
                'computation_time': None,
                'success': False
            })

    return results

# Run comparison study
print("=== Large-Scale Comparison Study ===")

# Generate multiple datasets
datasets = []
dataset_configs = [
    {'n': 100, 'confounding': 0.2, 'name': 'Small, Weak confounding'},
    {'n': 100, 'confounding': 0.8, 'name': 'Small, Strong confounding'},
    {'n': 1000, 'confounding': 0.2, 'name': 'Large, Weak confounding'},
    {'n': 1000, 'confounding': 0.8, 'name': 'Large, Strong confounding'},
]

all_results = []
for i, config in enumerate(dataset_configs):
    print(f"Analyzing dataset {i+1}: {config['name']}")

    X, Y = generate_dataset(
        n=config['n'],
        confounding_strength=config['confounding'],
        seed=42 + i
    )

    dataset_results = analyze_dataset(X, Y, i+1)

    # Add dataset metadata
    for result in dataset_results:
        result.update({
            'sample_size': config['n'],
            'confounding_strength': config['confounding'],
            'dataset_name': config['name']
        })

    all_results.extend(dataset_results)

# Compile results
df_results = pd.DataFrame(all_results)

# Summary statistics
print("\\n=== Summary Results ===")
summary = df_results[df_results['success']].groupby(['algorithm', 'confounding_strength']).agg({
    'width': ['mean', 'std'],
    'computation_time': ['mean', 'std']
}).round(4)

print(summary)

# Best performing algorithm by scenario
print("\\n=== Best Algorithm by Scenario (narrowest bounds) ===")
best_by_scenario = df_results[df_results['success']].loc[
    df_results[df_results['success']].groupby(['dataset_id'])['width'].idxmin()
][['dataset_name', 'algorithm', 'width']]

print(best_by_scenario.to_string(index=False))

=== Large-Scale Comparison Study ===
Analyzing dataset 1: Small, Weak confounding


Analyzing dataset 2: Small, Strong confounding


ValueError: p < 0, p > 1 or p contains NaNs

In [5]:
def safe_compute_bounds(scenario, algorithm_name, query='ATE', **kwargs):
    """Safely compute bounds with fallback."""
    try:
        dispatcher = getattr(scenario, query)
        algorithm = getattr(dispatcher, algorithm_name)
        return algorithm(**kwargs)
    except (AttributeError, ImportError) as e:
        print(f"Algorithm {algorithm_name} not available: {e}")
        return None

In [35]:
Z = np.array([0, 1, 1, 0, 1])  # Example instrumental variable data
X = np.array([0, 1, 1, 0, 1])  # Example treatment data
Y = np.array([1, 0, 0.1, 0.5, 0.7])  # Example outcome data



scenario = ContIV(X, Y, Z) # Instantiate the scenario with the data
scenario.ATE.zhangbareinboim() # Call the Zaffalon bounds method

(np.float64(-0.48333333), np.float64(-0.48333333))